In [15]:
# LangchainHub 사용을 위한 패키지 설치 및 환경변수 설정
# !pip install langchainhub python-dotenv

import os
from dotenv import load_dotenv
from langchain import hub

# .env 파일에서 환경변수 로드
load_dotenv()

# LANGCHAIN_API_KEY 환경변수 값 확인
langchain_api_key = os.getenv('LANGCHAIN_API_KEY')
print(f"LANGCHAIN_API_KEY가 설정되어 있나요?: {langchain_api_key[:10]}...")


LANGCHAIN_API_KEY가 설정되어 있나요?: [본인계정으로 발급...


In [16]:
# 1. DoucmentLoader
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("data/[AI.GOV_해외동향]_2025-1호.pdf")

docs = loader.load()

print(f"1번째 문서 내용 : {docs[5].page_content}")

1번째 문서 내용 : 6
그림 의 자율 에이전트 프레임워크1. Antropic (Autonomous agent) 
주요 에이전트 서비스 AI Operator (OpenAI, '25.1.)② Ⅳ
 ▸ 는 가 웹 인터페이스와 직접 상호작용하는 서비스인 공개OpenAI AI ‘Operator’ 5)
   - 의 비전 기능과 강화 학습을 결합한 컴퓨터 사용 에이전트 모델을 활용GPT-4o (CUA) 
하여 버튼 메뉴 및 텍스트 필드 등 그래픽 사용자 인터페이스 와 상호작용 구현, (GUI)
 ▸ 는 에 쿼리 입력 결과 분석 예약 플랫폼 탐색 등 인간의 브라우징Operator Google , ,  
패턴을 모방하여 호텔 예약 양식 제출과 같은 구조화된 작업 자동화
   - 사용자의 명령을 받으면 는 가 제공하는 원격 가상 브라우저를 열고Operator OpenAI , 
사람이 화면을 보듯 스크린샷으로 를 인식한 뒤 마우스 클릭 키보드 입력 스크롤 UI , , 
등의 행동을 자동화 다만 구조화되지 않은 비표준적인 웹 환경에서는 활용이 제한. , 
 ▸ 는 월부터 오스트레일리아 브라질 캐나다 인도 일본 싱가포르 한국OpenAI 2 , , , , , , , 
영국 등 개국에서 서비스를 출시8 ‘Operator’ 
   - 는 현재 연구 미리보기 상태로 공개되었으며 공식 블로그‘Operator’ (Research Preview) , 
에서 도어대시 우버 등과 협력한 서비스를 개발하고 있다고 발표(DoorDash), (Uber) 
 ▸국내에서는 카카오 및 야놀자와 제휴하여 카카오톡 쇼핑과 야놀자 예약 업무를 
로 수행 가능Operator
   - 스스로 카카오톡 선물하기 서비스에 접속해 이용자에게 맞춤형 구매를 제안하거나 야놀자‘ ’ , ‘  
숙박 항공 예약 서비스에서 날씨 지역 기간 등 사용자의 조건에 맞춰 숙박 예약 가능· ’ , , 
 ▸ 년 월 는 컴퓨터 사용 에이전트 모델을 로 공개'25 3 , OpenAI (CUA) API
   - 외부 개발자들도 를 활용하여 자체

In [17]:
# 2. TextSplitter
from langchain_text_splitters import CharacterTextSplitter

# 기본 설정으로 텍스트 분할기 생성
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=200,     # 각 청크 최대 200자
    chunk_overlap=20   # 20자씩 겹침
)

split_documents = text_splitter.split_documents(docs)

print(f"분할된 청크의수: {len(split_documents)}")

분할된 청크의수: 131


In [18]:
# 3. Embedding
from langchain_openai import OpenAIEmbeddings

# 1. OpenAI 임베딩 모델 초기화 (ChatGPT-4o-mini와 호환)
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",  # OpenAI의 최신 임베딩 모델
    # API 키는 환경변수 OPENAI_API_KEY에서 자동으로 읽어옵니다
)

print(f"모델명: {embeddings.model}")

모델명: text-embedding-3-small


In [19]:
# 4. VectorStore
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [20]:
# 5. Retriever
retriever = db.as_retriever()

In [21]:
# 6. Prompt
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """당신은 질문에 답변하는 작업을 수행하는 어시스턴트입니다.
다음에 제공된 문맥 정보를 바탕으로 질문에 답하세요.
정답을 모를 경우, 모른다고만 말하세요.
답변은 반드시 한국어로 작성하세요.

#문맥:
{context}

#질문:
{question}

#답변:"""
)

In [22]:
# 7. Model
from langchain.chat_models import init_chat_model

# 문맥기반으로 대답을 해야하기 때문에 창의성 0으로 설정
llm = init_chat_model("gpt-4o-mini", model_provider="openai", temperature=0.0)

In [23]:
# 8. Chain
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(), # 다음 체인으로 값을 그대로 넘김
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# 9. 테스트
query = "주요 해외 AI 에이전트 서비스 동향에 대해 알려줘"
response = chain.invoke(query)

print(response)

Failed to get info from https://api.smith.langchain.com: LangSmithError('Failed to GET /info in LangSmith API. latin-1\n[본인계정으로 발급받은 API KEY를 작성하세요...]\n1\n7\nordinal not in range(256)')
Failed to batch ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. latin-1
[본인계정으로 발급받은 API KEY를 작성하세요...]
1
7
ordinal not in range(256)
post: trace=0f0d5882-f14f-4c71-bc13-51aa523a4fa9,id=0f0d5882-f14f-4c71-bc13-51aa523a4fa9; trace=0f0d5882-f14f-4c71-bc13-51aa523a4fa9,id=311830f5-37d9-4c4e-9cd1-cae8c01f8e1c; trace=0f0d5882-f14f-4c71-bc13-51aa523a4fa9,id=bcad7e1a-6211-4c58-ab75-0beed8d54699; trace=0f0d5882-f14f-4c71-bc13-51aa523a4fa9,id=bd25d868-5979-4edb-bbbf-e7edf57d2628
Failed to batch ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. latin-1
[본인계정으로 발급받은 API KEY를 작성하세요...]
1
7
ordinal not in range(256)
post: trace=0f0d5882-f14f-4c71-bc13-51aa523a4fa9,id=63b1ef

주요 해외 AI 에이전트 서비스 동향에 따르면, 2025년에는 다양한 에이전트 서비스가 출시되고 있으며, 특히 'Computer Use', 'Operator', 'Manus AI'와 같은 서비스가 주목받고 있습니다. 이 중 Manus AI는 중국의 스타트업이 개발한 자율형 인공지능 에이전트로, 인간의 개입 없이 복잡한 작업을 독립적으로 수행할 수 있도록 설계되었습니다. 이러한 에이전트들은 인공지능 산업에서 중요한 트렌드로 자리잡고 있습니다.


Failed to batch ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. latin-1
[본인계정으로 발급받은 API KEY를 작성하세요...]
1
7
ordinal not in range(256)
post: trace=0f0d5882-f14f-4c71-bc13-51aa523a4fa9,id=060fa01c-3495-4fcc-b221-d5e3e502547c; trace=562cbee6-add0-406e-8416-68c3a5fc8050,id=562cbee6-add0-406e-8416-68c3a5fc8050; patch: trace=0f0d5882-f14f-4c71-bc13-51aa523a4fa9,id=0f0d5882-f14f-4c71-bc13-51aa523a4fa9; trace=0f0d5882-f14f-4c71-bc13-51aa523a4fa9,id=9e72aa83-dd4f-4ed4-b4f1-c01e5fb22c8f
Failed to batch ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. latin-1
[본인계정으로 발급받은 API KEY를 작성하세요...]
1
7
ordinal not in range(256)
patch: trace=562cbee6-add0-406e-8416-68c3a5fc8050,id=562cbee6-add0-406e-8416-68c3a5fc8050


In [25]:
llm.invoke(query)

AIMessage(content='2023년 현재, 해외 AI 에이전트 서비스는 다양한 분야에서 빠르게 발전하고 있으며, 여러 주요 동향이 나타나고 있습니다. 다음은 그 주요 동향입니다:\n\n1. **개인화된 사용자 경험**: AI 에이전트는 사용자 데이터를 분석하여 개인 맞춤형 서비스를 제공하는 데 중점을 두고 있습니다. 예를 들어, 추천 시스템이나 개인화된 콘텐츠 제공이 강화되고 있습니다.\n\n2. **자연어 처리(NLP) 기술의 발전**: GPT-3와 같은 고급 자연어 처리 모델이 상용화되면서, AI 에이전트는 더욱 자연스럽고 인간과 유사한 대화를 할 수 있게 되었습니다. 이는 고객 서비스, 상담, 교육 등 다양한 분야에서 활용되고 있습니다.\n\n3. **멀티모달 AI**: 텍스트, 이미지, 음성 등 다양한 형태의 데이터를 동시에 처리할 수 있는 멀티모달 AI 기술이 발전하고 있습니다. 이는 사용자와의 상호작용을 더욱 풍부하게 만들어 줍니다.\n\n4. **AI 윤리 및 규제**: AI 기술의 발전과 함께 윤리적 문제와 규제에 대한 논의도 활발해지고 있습니다. 데이터 프라이버시, 알고리즘의 편향성 문제 등이 주요 이슈로 떠오르고 있으며, 이에 대한 해결책을 모색하는 노력이 진행되고 있습니다.\n\n5. **AI 기반 자동화**: 비즈니스 프로세스 자동화(BPA)와 로보틱 프로세스 자동화(RPA) 분야에서 AI 에이전트의 활용이 증가하고 있습니다. 이는 기업의 효율성을 높이고 비용을 절감하는 데 기여하고 있습니다.\n\n6. **AI 에이전트의 통합**: 다양한 플랫폼과 서비스에 AI 에이전트를 통합하는 경향이 있습니다. 예를 들어, CRM 시스템, 헬스케어, 금융 서비스 등 여러 산업에서 AI 에이전트를 활용하여 고객 경험을 개선하고 있습니다.\n\n7. **AI 에이전트의 협업**: 인간과 AI 에이전트 간의 협업이 강조되고 있습니다. AI는 반복적이고 단순한 작업을 처리하고, 인간은 창의적이고 복잡한 문제를 해결하는 방식으로 역할이 분담되고 있습